In [1]:
# get data
from urllib.request import urlopen, Request

# parse data from FinViz 
from bs4 import BeautifulSoup
import os
# manipulate and store the data in DataFrames
import pandas as pd
# plot the sentiment on a chart
import matplotlib.pyplot as plt
%matplotlib inline

# NLTK VADER for sentiment analysis on the news headlines
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
# take the raw part of the url, we will append the ticker to the end of the link to pull up its data
# Extract data from finviz use the raw url.
finwiz_url = 'https://finviz.com/quote.ashx?t='


hardcode using any ticker for proof of concept then make dynamic on streamlit.
on streamlit: 
   - change aapl to an f string  so end user can dyanmically choose their tickers.
   -  use st.multiselect() so user can pick several tickers as once as a comparison.
   
(Starts to get ugly after 10. tickers. 3 - 5 is cleanest so far)

In [3]:
# Initiate an empty dictionary to hold the news tables from website
news_tables = {}
tickers = ['AAPL']

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}) 
    response = urlopen(req)    
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response)
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')
    # Add the table to our dictionary
    news_tables[ticker] = news_table

In [4]:
# <td> tag defines the standard cells in the table which are displayed as normal-weight, left-aligned text. 
# <tr> tag defines the table rows.

# Read headlines for 'AAPL' 
aapl = news_tables['AAPL']
# Get all the table rows tagged in HTML with <tr> into 'aapl_tr'
aapl_tr = aapl.findAll('tr')

#Create a list to append the titles
title_list = []
        
# The enumerate() function takes a collection and returns it as an enumerate object.
for index, table_row in enumerate(aapl_tr):
    # Read the text of the element 'a' (Anchor tag) into 'title' 
    title = table_row.a.text
    # Read the text of the element 'td' into 'timestamp' for the timestamp
    timestamp = table_row.td.text.split()
    
    # if the length of 'timestamp' is 1, load 'time' as the only element
    if len(timestamp) == 1:
        time = timestamp[0]
            
    # else load 'date' as the 1st element and 'time' as the second    
    else:
        date = timestamp[0]
        time = timestamp[1]
    ticker = tickers
            
    # Append the title and timestamp to list format. 
    title_list.append([tickers, date, time, title])
    # Create a dataframe using the 'title_list'
    finviz_headlines = pd.DataFrame(title_list, columns=[['ticker', 'date', 'time', 'title']])
finviz_headlines

,ticker,date,time,title
0,[AAPL],Aug-11-22,11:28AM,NFL should be careful about streaming deals wi...
1,[AAPL],Aug-11-22,11:09AM,Is AppLovin Stock a Buy Now?
2,[AAPL],Aug-11-22,10:00AM,3 Reasons to Buy Apple Stock Now
3,[AAPL],Aug-11-22,09:58AM,Rivian Leads EV Slump After Investors Shun Mon...
4,[AAPL],Aug-11-22,09:51AM,Apple App Store sees no growth in July for fir...
...,...,...,...,...
95,[AAPL],Aug-05-22,06:50PM,Weekly Roundup
96,[AAPL],Aug-05-22,05:20PM,Advertising Is Still Going Strong. Apple Wants...
97,[AAPL],Aug-05-22,05:02PM,President Xi may use Speaker Pelosi's visit to...
98,[AAPL],Aug-05-22,04:45PM,The Best Multiport Wall Adapters of 2022


In [5]:
finviz_headlines.title

,title
0,NFL should be careful about streaming deals wi...
1,Is AppLovin Stock a Buy Now?
2,3 Reasons to Buy Apple Stock Now
3,Rivian Leads EV Slump After Investors Shun Mon...
4,Apple App Store sees no growth in July for fir...
...,...
95,Weekly Roundup
96,Advertising Is Still Going Strong. Apple Wants...
97,President Xi may use Speaker Pelosi's visit to...
98,The Best Multiport Wall Adapters of 2022


# Sentiment Analysis

In [6]:
# exmaple, not needed for code
analyzer = SentimentIntensityAnalyzer().polarity_scores(finviz_headlines['title'])
# analyzer

AttributeError: 'DataFrame' object has no attribute 'encode'

In [27]:
vader = SentimentIntensityAnalyzer()
f = lambda title: vader.polarity_scores(title)['compound']
finviz_headlines['compound'] = finviz_headlines['title'].apply(f)
finviz_headlines

AttributeError: 'Series' object has no attribute 'encode'

In [19]:
# Percentage 
def percentage(part,whole):
    return 100 * float(part)/float(whole)


# Initialize Values
positive = 0
negative = 0
neutral = 0


# Initialize open buckets as holder for list
title_list = []
neutral_list = []
negative_list = []
positive_list = []

# Iterating over the titles in the dataframe
for title in finviz_headlines['title']:
    title_list.append(title)
    analyzer = SentimentIntensityAnalyzer().polarity_scores(title)
    neg = analyzer['neg']
    neu = analyzer['neu']
    pos = analyzer['pos']
    comp = analyzer['compound']

    if neg > pos:
        # append the headline that satisfies 'negative_list' conditions
        negative_list.append(title)
        #increases the count by 1
        negative += 1 
         # if sentiment is negative, call it -1 and append it to the sentiment_list
        sentiment_list.append(-1)
        sentiment += 1
    elif pos > neg: 
        # append the tweet that satisfies 'positive_list' conditions
        positive_list.append(title)
        #increase the count by 1
        positive += 1 
         # if sentiment is positive, call it 1 and append it to the sentiment_list
        sentiment_list.append(1)
        sentiment += 1
    elif pos == neg:
        # append the tweet that satisfies 'neutral_list' conditions
        neutral_list.append(title) 
        #increase the count by 1 
        neutral += 1 

AttributeError: 'tuple' object has no attribute 'encode'

In [9]:
# Percent Positive
positive = percentage(positive, len(finviz_headlines)) 
# Percent Negative 
negative = percentage(negative, len(finviz_headlines)) 
# Percent Neutral
neutral = percentage(neutral, len(finviz_headlines)) 

In [11]:
#Convert the lists to pandas dataframe 'title_list', 'neutral_list', 'negative_list', 'positive_list'
title_list = pd.DataFrame(title_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)

In [13]:
# print("Since " + number_of_days + " days, there have been", len(tweet_list1) ,  "tweets on " + query,'\n*')
print("Positive Sentiment:", '%.2f' % len(positive_list),'\n*')
print("Neutral Sentiment:", '%.2f' % len(neutral_list), '\n*')
print("Negative Sentiment:", '%.2f' % len(negative_list), '\n*')

Positive Sentiment: 0.00 
*
Neutral Sentiment: 0.00 
*
Negative Sentiment: 0.00 
*


In [14]:
#Create the PieCart
labels = ['Positive ['+str(round(positive))+'%]' , 'Neutral ['+str(round(neutral))+'%]','Negative ['+str(round(negative))+'%]']
sizes = [positive, neutral, negative]
colors = ['yellowgreen', 'blue','red']
patches, texts = plt.pie(sizes,colors=colors, startangle=90)
plt.style.use('default')
plt.legend(labels)
plt.title("Sentiment Analysis Result for keyword= "+query+"" )
plt.axis('equal')
plt.show()
st.pyplot(plt)

C:\Users\Owner\AppData\Roaming\Python\Python37\site-packages\matplotlib\axes\_axes.py:3050: RuntimeWarning: invalid value encountered in true_divide
  x = x / sx


ValueError: cannot convert float NaN to integer

ValueError: need at least one array to concatenate

<Figure size 432x288 with 1 Axes>